In [113]:
#import classes
from random import shuffle
import gensim
import os
import collections
import smart_open
import random
import re

In [114]:
def read_labels(fname):
    with smart_open.smart_open(fname) as f:
        for line in enumerate(f):
            yield line

In [115]:
train_labels = list(read_labels("labels-train.txt"))
test_labels = list(read_labels("labels-test.txt"))

In [116]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), train_labels[i][1].decode('utf-8'))

In [117]:
train_corpus = list(read_corpus("smoking-train.txt"))
test_corpus = list(read_corpus("smoking-test.txt",tokens_only=True))
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)
model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=100)

CPU times: user 1min 18s, sys: 1.25 s, total: 1min 19s
Wall time: 30.9 s


22801624

In [118]:
model.most_similar('quit')

[('unemployed', 0.7107837200164795),
 ('smoking', 0.6728432774543762),
 ('pipe', 0.6254364252090454),
 ('drink', 0.615814745426178),
 ('cigarette', 0.6044490933418274),
 ('tobacco', 0.6023449897766113),
 ('lives', 0.601466178894043),
 ('alcohol', 0.5989133715629578),
 ('detail', 0.5954065322875977),
 ('ethanol', 0.5915155410766602)]

In [119]:
model.most_similar('smoking')

[('cigarette', 0.6794581413269043),
 ('quit', 0.6728431582450867),
 ('ethanol', 0.632678210735321),
 ('alcohol', 0.6206820011138916),
 ('tobacco', 0.6161278486251831),
 ('drink', 0.5640317797660828),
 ('obesity', 0.5633103847503662),
 ('drinks', 0.5615329146385193),
 ('smoked', 0.552899956703186),
 ('rh', 0.5519952774047852)]

In [120]:
train_vecs = model.docvecs
test_vecs = list([])
for elem in test_corpus:
    a = list(model.infer_vector(elem))
    test_vecs.append(a)